In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
from models import EMGConvNet

# === 1. Define Paths ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === 2. Load Data ===
X = np.load(X_path)   # shape: (5, 8, 230000)
y = np.load(y_path)

# === 3. Extract windows ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]  # label = pose at last time step
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === 4. Keep only sessions 0, 1, 2, 3 for cross-validation ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

# === 5. Define Dataset class ===
class EMGDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, standardize=True):
        self.standardize = standardize
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if self.standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# === 6. Perform 4-fold Cross-Validation ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for fold in range(4):
    print(f"\n===== Fold {fold+1} =====")
    
    # Build train/val split based on sessions
    X_train = np.vstack([X_sessions_cv[i] for i in range(4) if i != fold])
    y_train = np.vstack([y_sessions_cv[i] for i in range(4) if i != fold])
    
    X_val = X_sessions_cv[fold]
    y_val = y_sessions_cv[fold]
    
    train_dataset = EMGDataset(X_train, y_train)
    val_dataset = EMGDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)

    # Build a new model for each fold
    model = EMGConvNet(
        conv_layers_config=[(16, 5, 1), (32, 5, 2), (64, 3, 2)],
        fc_layers_config=[768, 512, 256, 128, 64],
        output_dim=51,
        verbose=False
    )
    model.build()
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    def compute_rmse(y_pred, y_true):
        return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

    for epoch in range(100):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)

        model.eval()
        val_loss = 0.0
        val_rmse = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                rmse = compute_rmse(outputs, y_batch)
                val_loss += loss.item() * X_batch.size(0)
                val_rmse += rmse.item() * X_batch.size(0)

        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        val_rmse /= len(val_loader.dataset)

        print(f"Epoch {epoch+1:2d} | Train MSE: {train_loss:.4f} | Val MSE: {val_loss:.4f} | Val RMSE: {val_rmse:.4f}")



===== Fold 1 =====
Epoch  1 | Train MSE: 241.7941 | Val MSE: 155.4830 | Val RMSE: 12.3920
Epoch  2 | Train MSE: 151.5756 | Val MSE: 160.7641 | Val RMSE: 12.6020
Epoch  3 | Train MSE: 145.2440 | Val MSE: 135.8437 | Val RMSE: 11.6113
Epoch  4 | Train MSE: 139.4539 | Val MSE: 151.6866 | Val RMSE: 12.2461
Epoch  5 | Train MSE: 143.7375 | Val MSE: 134.7240 | Val RMSE: 11.5748
Epoch  6 | Train MSE: 133.5207 | Val MSE: 136.7456 | Val RMSE: 11.6307
Epoch  7 | Train MSE: 97.5962 | Val MSE: 88.1232 | Val RMSE: 9.2507
Epoch  8 | Train MSE: 73.0904 | Val MSE: 68.9420 | Val RMSE: 8.2315
Epoch  9 | Train MSE: 63.2051 | Val MSE: 70.3549 | Val RMSE: 8.2774
Epoch 10 | Train MSE: 55.3050 | Val MSE: 66.1819 | Val RMSE: 8.0689
Epoch 11 | Train MSE: 56.2595 | Val MSE: 61.7439 | Val RMSE: 7.7851
Epoch 12 | Train MSE: 52.2481 | Val MSE: 61.9122 | Val RMSE: 7.7945
Epoch 13 | Train MSE: 51.6163 | Val MSE: 65.6466 | Val RMSE: 8.0220
Epoch 14 | Train MSE: 52.2411 | Val MSE: 63.3619 | Val RMSE: 7.8810
Epoch 15 |

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
from pathlib import Path
from models import EMGConvNet

# === 1. Define Paths ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === 2. Load Data ===
X = np.load(X_path)  # shape: (5, 8, 230000)
y = np.load(y_path)

# === 3. Extract windows by session ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]  # label = pose at last time step
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === 4. Keep only sessions 0,1,2,3 for cross-validation ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

# === 5. Define Dataset class ===
class EMGDataset(Dataset):
    def __init__(self, X, y, standardize=True):
        self.standardize = standardize
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if self.standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# === 6. Define architecture config separately ===
architecture_config = {
    "conv_layers": [(16, 5, 1), (32, 5, 2), (64, 3, 2)],
    "fc_layers": [768, 512, 256, 128, 64],
    "conv_dropouts": [0, 0, 0],
    "fc_dropouts": [0, 0, 0, 0, 0]
}

experiment_log = {
    "architecture": architecture_config,
    "folds": []
}

# === 7. Perform 4-fold Cross-Validation ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for fold_idx in range(4):
    print(f"\n===== Fold {fold_idx+1} =====")
    fold_log = {"fold_number": fold_idx, "epochs": []}

    # Build train/val split based on sessions
    X_train = np.vstack([X_sessions_cv[i] for i in range(4) if i != fold_idx])
    y_train = np.vstack([y_sessions_cv[i] for i in range(4) if i != fold_idx])
    X_val = X_sessions_cv[fold_idx]
    y_val = y_sessions_cv[fold_idx]

    train_dataset = EMGDataset(X_train, y_train)
    val_dataset = EMGDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)

    model = EMGConvNet(
        conv_layers_config=architecture_config["conv_layers"],
        fc_layers_config=architecture_config["fc_layers"],
        conv_dropouts=architecture_config["conv_dropouts"],
        fc_dropouts=architecture_config["fc_dropouts"],
        output_dim=51,
        verbose=False
    )
    model.build()
    model.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    def compute_rmse(y_pred, y_true):
        return torch.sqrt(torch.mean((y_pred - y_true) ** 2))

    for epoch in range(100):
        model.train()
        train_loss = 0.0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)

        model.eval()
        val_loss = 0.0
        val_rmse = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                rmse = compute_rmse(outputs, y_batch)
                val_loss += loss.item() * X_batch.size(0)
                val_rmse += rmse.item() * X_batch.size(0)

        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        val_rmse /= len(val_loader.dataset)

        epoch_log = {
            "epoch": epoch + 1,
            "train_mse": train_loss,
            "val_mse": val_loss,
            "val_rmse": val_rmse
        }
        fold_log["epochs"].append(epoch_log)

        print(f"Epoch {epoch+1:2d} | Train MSE: {train_loss:.4f} | Val MSE: {val_loss:.4f} | Val RMSE: {val_rmse:.4f}")

    experiment_log["folds"].append(fold_log)

# === 8. Save experiment log ===

log_path = Path("experiment_log.json")

if log_path.exists():
    with open(log_path, "r") as f:
        existing_data = json.load(f)
else:
    existing_data = []

existing_data.append(experiment_log)

with open(log_path, "w") as f:
    json.dump(existing_data, f, indent=4)



===== Fold 1 =====
Epoch  1 | Train MSE: 270.1534 | Val MSE: 195.7387 | Val RMSE: 13.8879
Epoch  2 | Train MSE: 157.6006 | Val MSE: 183.0145 | Val RMSE: 13.4642
Epoch  3 | Train MSE: 149.2386 | Val MSE: 141.4410 | Val RMSE: 11.8324
Epoch  4 | Train MSE: 140.5110 | Val MSE: 134.3813 | Val RMSE: 11.5557
Epoch  5 | Train MSE: 138.6086 | Val MSE: 160.2620 | Val RMSE: 12.5875
Epoch  6 | Train MSE: 139.8710 | Val MSE: 143.3964 | Val RMSE: 11.9104
Epoch  7 | Train MSE: 117.8616 | Val MSE: 93.7630 | Val RMSE: 9.6233
Epoch  8 | Train MSE: 72.7861 | Val MSE: 68.3536 | Val RMSE: 8.1907
Epoch  9 | Train MSE: 63.3437 | Val MSE: 63.1300 | Val RMSE: 7.8655
Epoch 10 | Train MSE: 54.1293 | Val MSE: 67.7624 | Val RMSE: 8.1257
Epoch 11 | Train MSE: 57.4323 | Val MSE: 82.0178 | Val RMSE: 8.8900
Epoch 12 | Train MSE: 60.2490 | Val MSE: 60.6463 | Val RMSE: 7.7070
Epoch 13 | Train MSE: 53.0669 | Val MSE: 64.3633 | Val RMSE: 7.9419
Epoch 14 | Train MSE: 53.3276 | Val MSE: 67.4614 | Val RMSE: 8.0937
Epoch 15 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
from pathlib import Path
from models import EMGConvNet, TrainingManager, CrossValidationManager
from utils import save_experiment_log

In [2]:
# === Paths to data ===
X_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_X.npy")
y_path = Path(r"D:\Uni\F422\F422\F422 EMG project data\guided\guided_dataset_y.npy")

# === Load Data ===
X = np.load(X_path)   # (5, 8, 230000)
y = np.load(y_path)   # (5, 51, 230000)

# === Extract windows by session ===
def extract_windows_by_session(X, y, window_size=500, stride=250):
    X_sessions, y_sessions = [], []
    sessions = X.shape[0]

    for sess in range(sessions):
        X_windows, y_targets = [], []
        for start in range(0, X.shape[2] - window_size + 1, stride):
            end = start + window_size
            x_window = X[sess, :, start:end]
            y_target = y[sess, :, end-1]
            X_windows.append(x_window)
            y_targets.append(y_target)
        X_sessions.append(np.stack(X_windows))
        y_sessions.append(np.stack(y_targets))

    return X_sessions, y_sessions

X_sessions, y_sessions = extract_windows_by_session(X, y)

# === Keep only first 4 sessions for CV ===
X_sessions_cv = X_sessions[:4]
y_sessions_cv = y_sessions[:4]

class EMGDataset(Dataset):
    def __init__(self, X, y, standardize=True):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

        if standardize:
            self.mean = self.X.mean(dim=(0, 2), keepdim=True)
            self.std = self.X.std(dim=(0, 2), keepdim=True)
            self.X = (self.X - self.mean) / (self.std + 1e-8)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


In [3]:
def save_experiment_log(log, path="logs/experiment_log.json"):
    path = Path(path)
    path.parent.mkdir(parents=True, exist_ok=True)

    if path.exists():
        with open(path, "r") as f:
            existing = json.load(f)
    else:
        existing = []

    existing.append(log)

    with open(path, "w") as f:
        json.dump(existing, f, indent=4)


In [4]:
# Model and training configs
model_config = {
    "conv_layers_config": [(16, 5, 1), (32, 5, 2)],
    "fc_layers_config": [512, 256, 128, 64],
    "output_dim": 51,
    "verbose": False
}

training_config = {
    "lr": 1e-3,
    "epochs": 100,
    "batch_size": 64,
    "log_every": 1
}

# Cross-validation runner
cross_validator = CrossValidationManager(
    model_class=EMGConvNet,
    model_config=model_config,
    data=X_sessions_cv,
    labels=y_sessions_cv,
    training_config=training_config,
    dataset_class=EMGDataset,
    dataset_config={"standardize": True},
    n_folds=4
)

# Run and save
experiment_log = cross_validator.run()
save_experiment_log(experiment_log, path="logs/neuralnetwork_log.json")



===== Fold 1/4 =====
Epoch   1 | Train MSE: 260.1710 | Val MSE: 171.0860 | Val RMSE: 12.9763
Epoch   2 | Train MSE: 163.4468 | Val MSE: 152.0605 | Val RMSE: 12.2575
Epoch   3 | Train MSE: 150.4566 | Val MSE: 143.7922 | Val RMSE: 11.9183
Epoch   4 | Train MSE: 113.1277 | Val MSE: 96.6523 | Val RMSE: 9.7362
Epoch   5 | Train MSE: 77.1006 | Val MSE: 79.9829 | Val RMSE: 8.8508
Epoch   6 | Train MSE: 65.5053 | Val MSE: 90.2205 | Val RMSE: 9.3425
Epoch   7 | Train MSE: 59.6206 | Val MSE: 77.8296 | Val RMSE: 8.7337
Epoch   8 | Train MSE: 59.6210 | Val MSE: 87.4416 | Val RMSE: 9.1830
Epoch   9 | Train MSE: 58.8816 | Val MSE: 71.5518 | Val RMSE: 8.4012
Epoch  10 | Train MSE: 59.7890 | Val MSE: 69.7020 | Val RMSE: 8.3023
Epoch  11 | Train MSE: 55.9174 | Val MSE: 69.9906 | Val RMSE: 8.2959
Epoch  12 | Train MSE: 53.5833 | Val MSE: 65.7622 | Val RMSE: 8.0393
Epoch  13 | Train MSE: 54.0129 | Val MSE: 64.5510 | Val RMSE: 7.9809
Epoch  14 | Train MSE: 53.4739 | Val MSE: 61.6555 | Val RMSE: 7.7989
Ep